In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

train = pd.read_csv('../input/tabular-playground-series-jul-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-jul-2021/test.csv')
sample = pd.read_csv('../input/tabular-playground-series-jul-2021/sample_submission.csv')

In [ ]:
sample.info()

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
train.tail()

In [ ]:
test.head()

In [ ]:
train['date_time'] = pd.to_datetime(train['date_time'])

In [ ]:
i = -1
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize = (20,10))
for ax in (ax1, ax2, ax3):
    ax.plot(train['date_time'], train.iloc[:,i])
    ax.set_ylabel(train.columns[i])
    i -= 1

In [ ]:
y_train = train.iloc[:,-3:]

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

diff_targets = y_train.diff(periods=1)
fig, axs = plt.subplots(3, 3, figsize = (20,20))
for i in range(3):
    diff = diff_targets.iloc[1:, i]
    axs[i, 0].plot(diff, label = diff_targets.columns[i])
    axs[i, 0].set_title(y_train.columns[i])
    plot_acf(diff, ax = axs[i, 1])
    plot_pacf(diff, ax = axs[i, 2])
plt.show()

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

arima_model = ARIMA(y_train.iloc[:,0].values, order = (2,1,2))
model_fit = arima_model.fit(trend = 'nc',full_output = True, disp = True)
model_fit.summary()

In [ ]:
target_predicts = []
for i in range(3):
    model = ARIMA(y_train.iloc[:, i].values, order = (2,1,2))
    model_fit = model.fit(trend = 'nc', full_output = True, disp = True)
    target_pred = model_fit.forecast(len(test))
    target_predicts.append(target_pred[0])

In [ ]:
predictions = pd.DataFrame({sample.columns[0]:test['date_time'], sample.columns[1]:target_predicts[0], sample.columns[2]:target_predicts[1], sample.columns[3]:target_predicts[2]})

In [ ]:
predictions.head()

In [ ]:
predictions.to_csv('submission.csv', index = False)